In [2]:
import gzip
import pandas as pd

source_id = 2
target_ids = [31, 14,  3, 17, 47, 37]
id_name_map = {31: 'bindingdb', 14: 'faers', 3: 'pdb', 17: 'pharmgkb', 47: 'rxnorm', 37: 'brenda'}

filename_template = "drugbank_to_{db_name}.txt.gz"
# 生成URL列表
filenames = [filename_template.format(db_name=id_name_map[target_id]) for target_id in target_ids]

filenames.append('chembl_to_drugbank.txt.gz')
names1 = ['drugbank','target_db1']
names2 = ['drugbank','target_db2']


In [3]:
with gzip.open('./drug_mapping_data/drugbank_to_bindingdb.txt.gz', 'rt') as f:

    mapping_data = pd.read_csv(f, delimiter='\t', header=None,names=names1)

In [4]:
mapping_data.shape


(16581, 2)

In [5]:
mapping_data[mapping_data['target_db1']!='-'].shape

(5103, 2)

In [6]:
# 使用gzip模块解压缩并读取文件
with gzip.open('./drug_mapping_supplement_data/drugbank_to_bindingdb.txt.gz', 'rt') as f:
    
    supplement_data = pd.read_csv(f, delimiter='\t', header=1,names=names2)

In [7]:
supplement_data.head()

,drugbank,target_db2
0,DB04080,14676
1,DB16054,391587
2,DB12930,50170636
3,DB00432,50132298
4,DB04743,50056999


In [8]:
merged_data = pd.merge(mapping_data, supplement_data, on='drugbank', how='left')
merged_data.fillna('-1', inplace=True)
merged_data['target_db2'] = merged_data['target_db2'].astype(int)

In [9]:
merged_data

,drugbank,target_db1,target_db2
0,DB00001,-,-1
1,DB00002,-,-1
2,DB00003,-,-1
3,DB00004,-,-1
4,DB00005,-,-1
...,...,...,...
16824,DB18713,-,-1
16825,DB18714,-,-1
16826,DB18715,-,-1
16827,DB18716,-,-1


In [10]:
merged_data['target_db1'] = merged_data.apply(lambda row: row['target_db2'] if row['target_db1'] == '-' and row['target_db2']!=-1 else row['target_db1'], axis=1)

In [11]:
filtered_df = merged_data[(merged_data['target_db1'] == '-') & (merged_data['target_db2'] != -1)]
filtered_df

,drugbank,target_db1,target_db2


In [12]:
end_merged_data  = merged_data[['drugbank','target_db1']]


In [13]:
end_merged_data = end_merged_data.drop_duplicates(['drugbank'])
end_merged_data

,drugbank,target_db1
0,DB00001,-
1,DB00002,-
2,DB00003,-
3,DB00004,-
4,DB00005,-
...,...,...
16824,DB18713,-
16825,DB18714,-
16826,DB18715,-
16827,DB18716,-


In [14]:
end_merged_data[end_merged_data['target_db1']!='-'].shape

(5459, 2)